In [1]:
from nltk.translate.bleu_score import sentence_bleu

# Define your desired weights (example: higher weight for bi-grams)
weights = (0.25, 0.25, 0, 0)  # Weights for uni-gram, bi-gram, tri-gram, and 4-gram

# Reference and predicted texts (same as before)
reference = [["the", "picture", "is", "clicked", "by", "me"],
             ["this", "picture", "was", "clicked", "by", "me"]]
predictions = ["the", "picture", "the", "picture", "by", "me"]

# Calculate BLEU score with weights
score = sentence_bleu(reference, predictions, weights=weights)
print(score)

0.7186082239261684


/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [15]:
weights = (0.25, 0.25, 0, 0)  # Weights for uni-gram, bi-gram, tri-gram, and 4-gram
# Reference and predicted texts (same as before)
reference = ["The picture is clicked by me",
             "This picture was clicked by me"]
predictions = "The picture the picture by me"

# Calculate BLEU score with weights
score = sentence_bleu(reference, predictions, weights=weights)
score

0.8238134785437412

In [9]:
score

0.7186082239261684

In [11]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

reference = "The watermelon seeds pass through your digestive system"
candidate = "The seeds are digested like other fruit parts and typically pass through your system without causing issues, unless they are coated with pesticides."

# Tokenize by splitting (simple version)
ref_tokens =[reference.split()]
cand_tokens = candidate.split()

# Use a smoothing function because long answers rarely have exact n-gram matches
smooth = SmoothingFunction().method1

score = sentence_bleu(
    ref_tokens,
    cand_tokens,
    smoothing_function=smooth,
    weights=(1, 0, 0, 0 )  # BLUE-1
)

print(score)


0.2608695652173913


In [27]:
import re

def filter_deepseek(text: str) -> str:
    # Remove <think>...</think> and return only actual response
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    return text.strip()

s = """
The watermelon seeds pass through your digestive system.
"""

print(filter_deepseek(s))


The watermelon seeds pass through your digestive system.


In [ ]:
# Install dependencies if needed
# !pip install langchain ragas nest_asyncio

import nest_asyncio
import asyncio
from langchain_ollama import ChatOllama
from ragas.metrics.collections import BleuScore

# Allow async in Jupyter
nest_asyncio.apply()

# -----------------------------
# Ollama Chat model via LangChain
# -----------------------------
llm = ChatOllama(model="llama3.1", temperature=0.0)

# -----------------------------
# Async Ragas BLEU evaluation
# -----------------------------
async def eval_bleu(reference: str, query: str):
    """
    Evaluate Ollama response against reference using Ragas BLEU
    """
    prompt = f"Answer this question:\n{query}\nAnswer:"

    # Use LangChain ChatOllama to get response
    response = llm.predict(prompt)

    scorer = BleuScore(weights=(1, 0, 0, 0))
    result = await scorer.ascore(
        reference=reference,
        response=response
    )

    print("Query:", query)
    print("Reference:", reference)
    print("Generated:", response)
    print("BLEU Score:", result.value)
    print("-" * 50)

    return result.value

# -----------------------------
# Example usage
# -----------------------------
query = "Where is the Eiffel Tower located?"
reference = "The Eiffel Tower is located in Paris."

bleu_score = await eval_bleu(reference, query)
bleu_score

Query: Where is the Eiffel Tower located?
Reference: The Eiffel Tower is located in Paris.
Generated: Paris, France.
BLEU Score: 0.06988198185490689
--------------------------------------------------


0.06988198185490689

In [12]:
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer

# Example sentences
reference = ["the cat is on the mat"]
candidate = ["the cat is on mat"]

# # BLEU Score Calculation
# bleu = corpus_bleu(candidate, [reference])
# print(f"BLEU Score: {bleu.score}")

# ROUGE Score Calculation
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference[0], candidate[0])
print(f"ROUGE-1: {scores['rouge1']}")
print(f"ROUGE-L: {scores['rougeL']}")

ROUGE-1: Score(precision=1.0, recall=0.8333333333333334, fmeasure=0.9090909090909091)
ROUGE-L: Score(precision=1.0, recall=0.8333333333333334, fmeasure=0.9090909090909091)


In [3]:
from comet import download_model, load_from_checkpoint

# Load COMET model once (recommended: "Unbabel/wmt22-comet-da")
MODEL_PATH = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(MODEL_PATH)

def comet_score(hypotheses, references, sources=None):
    """
    Compute COMET scores.

    hypotheses: list[str]
    references: list[str]
    sources:    list[str] or None (many COMET models require sources)
    
    Returns: list of COMET scores (floats)
    """

    # Prepare batch
    data = []
    for h, r in zip(hypotheses, references):
        data.append({
            "src": "" if sources is None else sources,  # source can be empty string
            "mt": h,
            "ref": r,
        })

    # Compute scores
    model_output = model.predict(data, batch_size=8, gpus=0)
    return model_output.scores

hyps = [
    "The seeds pass through your body.",
    "Eating seeds is safe."
]

refs = [
    "The watermelon seeds pass through your digestive system.",
    "Eating watermelon seeds is harmless."
]

scores = comet_score(hyps, refs)
print(scores)
print("Average =", sum(scores)/len(scores))


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 70849.73it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/

[0.7281712889671326, 0.8200189471244812]
Average = 0.7740951180458069


In [1]:
from comet import download_model, load_from_checkpoint

# Choose your model from Hugging Face Hub
model_path = download_model("Unbabel/wmt22-comet-da")
# or for example:
# model_path = download_model("Unbabel/wmt22-comet-da")

# Load the model checkpoint:
model = load_from_checkpoint(model_path)

# Data must be in the following format:
data = [
    {
        "src": "10 到 15 分钟可以送到吗",
        "mt": "Can I receive my food in 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    },
    {
        "src": "Pode ser entregue dentro de 10 a 15 minutos?",
        "mt": "Can you send it for 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    }
]
# Call predict method:
model_output = model.predict(data, batch_size=8, gpus=1)

/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/markphan24/mas-llm/.venv/lib/python3.12/site-packages/torchmetrics/utilities/imports.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-12-07 12:50:42.819303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-07 12:50:42.995100: E exte

In [2]:
model_output

Prediction([('scores', [0.8417137265205383, 0.7745388746261597]),
            ('system_score', 0.808126300573349)])

In [3]:
f"""
            You are the final evaluator.
            Question: 
            Context:
            

            Other model responses:
      

            Produce the best single final answer. Keep it concise and accurate in 1 sentence.
            """.strip()

'You are the final evaluator.\n            Question: \n            Context:\n\n\n            Other model responses:\n\n\n            Produce the best single final answer. Keep it concise and accurate in 1 sentence.'